In [1]:
import numpy as np
import pandas as pd
import bokeh.plotting as bp
from bokeh.layouts import gridplot
#bp.output_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%run ../code/PNLEstimatorWrapper.py
%run ../code/EUtilities.py
EU = EUtilities()

In [3]:
#Se lee la señal anteriormente procesada
data_set = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_20s_EURUSD-2018-01.csv", parse_dates=True,
                      infer_datetime_format=True, index_col=0)

In [4]:
data_set.head()

,bid,ask
2018-01-01 23:58:00,1.20093,1.20115
2018-01-01 23:58:20,1.20082,1.20111
2018-01-01 23:58:40,1.20043,1.20110
2018-01-01 23:59:00,1.20015,1.20110
2018-01-01 23:59:20,1.20028,1.20110


In [16]:
window = 3
X, y_reg, bt = EU.build_dataset(data_set, window, binary_target=True, delete_constant_values=True, PNL=True)

In [11]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 20s ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/GaussianNB/pnl_20s_GNB_wsize3_2018-01.csv')

--------- GaussianNB 20s ----------------
average PNL :  -0.143961
average buy:  1348.0
average sell:  2336.2
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.13801   612.0  3000.0         190.0         2438.0
2018-01-08 -0.12600  3378.0   228.0        3010.0          228.0
2018-01-09 -0.11832  3676.0     0.0        3676.0            0.0
2018-01-10 -0.14112  3518.0   284.0        1986.0          220.0
2018-01-11 -0.13274  2270.0  1530.0        2270.0         1530.0
2018-01-12 -0.13557     0.0  3841.0           0.0         3841.0
2018-01-15 -0.16903  3842.0     0.0        3842.0            0.0
2018-01-16 -0.14234     0.0  3877.0           0.0         3877.0
2018-01-17 -0.16683     0.0  3998.0           0.0         3998.0
2018-01-18 -0.14779   144.0  3791.0          89.0         2976.0
2018-01-19 -0.16542  1626.0  2250.0         987.0         1842.0
2018-01-22 -0.14994  1813.0  2036.0        1142.0          495.0
2018-01

In [12]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,4,1,1))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('-------- KNeighborsClassifier 20s -------')
print('average PNL: ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_20s_KNC_wsize3_2018-01.csv')

-------- KNeighborsClassifier 20s -------
average PNL:  -0.142048
average buy:  1644.4
average sell:  2039.8
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.13652  1663.0  1949.0          14.0           18.0
2018-01-08 -0.13142  2940.0   666.0        1661.0           18.0
2018-01-09 -0.11841  2971.0   705.0        2263.0           23.0
2018-01-10 -0.13792  1790.0  2012.0          59.0           57.0
2018-01-11 -0.13646  1736.0  2064.0          17.0           29.0
2018-01-12 -0.14024   543.0  3298.0          27.0         2309.0
2018-01-15 -0.16142   526.0  3316.0          82.0         2276.0
2018-01-16 -0.14288  1872.0  2005.0          35.0           50.0
2018-01-17 -0.16368  1957.0  2041.0          18.0          123.0
2018-01-18 -0.15643  2192.0  1743.0          46.0           14.0
2018-01-19 -0.14803  1881.0  1995.0          12.0           14.0
2018-01-22 -0.13804  1931.0  1918.0          12.0           12.0
2018-01-

In [13]:
from sklearn.tree import DecisionTreeClassifier

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,4,1,1))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('------ DecisionTreeClassifier 20s -------')
print('average PNL: ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_20s_DTC_wsize3_2018-01.csv')

------ DecisionTreeClassifier 20s -------
average PNL:  -0.13886
average buy:  1784.6
average sell:  1899.6
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.13125  1709.0  1903.0          14.0           18.0
2018-01-08 -0.12886  2896.0   710.0        1670.0           11.0
2018-01-09 -0.11918  3050.0   626.0        2262.0           12.0
2018-01-10 -0.13969  1771.0  2031.0          34.0           42.0
2018-01-11 -0.14370  1861.0  1939.0          10.0           15.0
2018-01-12 -0.14167   698.0  3143.0          27.0         2317.0
2018-01-15 -0.15499   648.0  3194.0          42.0         2266.0
2018-01-16 -0.14206  1898.0  1979.0          15.0           13.0
2018-01-17 -0.16616  1945.0  2053.0          10.0           70.0
2018-01-18 -0.15524  2073.0  1862.0          31.0           10.0
2018-01-19 -0.14517  1891.0  1985.0          13.0           11.0
2018-01-22 -0.13701  1843.0  2006.0          11.0           12.0
2018-01-2

In [14]:
from sklearn.ensemble import RandomForestClassifier

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('------- RandomForestClassifier 20s -------')
print('PNL promedio: ', np.mean(result_RFC[:,0]))
print('buy promedio: ', np.mean(result_RFC[:,1]))
print('sell promedio: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_20s_RFC_wsize3_2018-01.csv')

------- RandomForestClassifier 20s -------
PNL promedio:  -0.1441505
buy promedio:  1664.9
sell promedio:  2019.3
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.13879  1716.0  1896.0          13.0           15.0
2018-01-08 -0.13390  2956.0   650.0        1670.0           11.0
2018-01-09 -0.12506  3031.0   645.0        2263.0           10.0
2018-01-10 -0.13848  1862.0  1940.0          19.0           42.0
2018-01-11 -0.12879  1812.0  1988.0          16.0           11.0
2018-01-12 -0.16354   582.0  3259.0          28.0         2332.0
2018-01-15 -0.16785   624.0  3218.0          76.0         2277.0
2018-01-16 -0.14804  1841.0  2036.0          18.0           32.0
2018-01-17 -0.15699  1937.0  2061.0          11.0           74.0
2018-01-18 -0.15246  2133.0  1802.0          17.0           11.0
2018-01-19 -0.16003  1882.0  1994.0          13.0           12.0
2018-01-22 -0.13533  1902.0  1947.0           9.0           14.0
201

### ahora probaremos con 30s

In [21]:
#Se lee la señal anteriormente procesada
data_set = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_30s_EURUSD-2018-01.csv", parse_dates=True,
                      infer_datetime_format=True, index_col=0)
window = 3
X, y_reg, bt = EU.build_dataset(data_set, window, binary_target=True, delete_constant_values=True, PNL=True)

In [17]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 30s ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/GaussianNB/pnl_30s_GNB_wsize3_2018-01.csv')

--------- GaussianNB 30s ----------------
average PNL :  -0.100517
average buy:  772.05
average sell:  1753.35
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.10094   691.0  1841.0         404.0         1223.0
2018-01-08 -0.08581  2394.0   130.0        2110.0          124.0
2018-01-09 -0.08742     0.0  2526.0           0.0         2526.0
2018-01-10 -0.09996  2135.0   482.0        1347.0          423.0
2018-01-11 -0.08292     0.0  2607.0           0.0         2607.0
2018-01-12 -0.09411   937.0  1689.0         468.0         1217.0
2018-01-15 -0.11704  2642.0     0.0        2642.0            0.0
2018-01-16 -0.09875     0.0  2644.0           0.0         2644.0
2018-01-17 -0.11831     0.0  2737.0           0.0         2737.0
2018-01-18 -0.10329   228.0  2470.0          83.0         2001.0
2018-01-19 -0.11852   794.0  1859.0         504.0         1406.0
2018-01-22 -0.10328   933.0  1703.0         747.0          475.0
2018-0

In [18]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,4,1,1))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('-------- KNeighborsClassifier 30s -------')
print('average PNL: ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_30s_KNC_wsize3_2018-01.csv')

-------- KNeighborsClassifier 30s -------
average PNL:  -0.097478
average buy:  1113.1
average sell:  1412.3
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.09386  1270.0  1262.0          14.0           20.0
2018-01-08 -0.08848  2033.0   491.0        1163.0           17.0
2018-01-09 -0.08591  2022.0   504.0        1552.0           29.0
2018-01-10 -0.09649  1179.0  1438.0          28.0           44.0
2018-01-11 -0.09679  1240.0  1367.0          16.0           13.0
2018-01-12 -0.09481   342.0  2284.0          12.0         1576.0
2018-01-15 -0.10683   232.0  2410.0          54.0         1560.0
2018-01-16 -0.09041  1096.0  1548.0          17.0           26.0
2018-01-17 -0.11512  1233.0  1504.0          11.0          172.0
2018-01-18 -0.10405  1389.0  1309.0          26.0           13.0
2018-01-19 -0.10174  1290.0  1363.0          10.0           20.0
2018-01-22 -0.09064  1314.0  1322.0          15.0           10.0
2018-01-

In [19]:
from sklearn.tree import DecisionTreeClassifier

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,4,1,1))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('------ DecisionTreeClassifier 30s -------')
print('average PNL: ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_30s_DTC_wsize3_2018-01.csv')

------ DecisionTreeClassifier 30s -------
average PNL:  -0.096987
average buy:  1301.65
average sell:  1223.75
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.09470  1269.0  1263.0          10.0           10.0
2018-01-08 -0.08966  2021.0   503.0        1163.0            7.0
2018-01-09 -0.08202  2000.0   526.0        1552.0           20.0
2018-01-10 -0.09939  1333.0  1284.0          25.0           47.0
2018-01-11 -0.09468  1280.0  1327.0          11.0           13.0
2018-01-12 -0.09203   385.0  2241.0          19.0         1576.0
2018-01-15 -0.11019  2068.0   574.0        1482.0           50.0
2018-01-16 -0.09550  1196.0  1448.0          13.0           23.0
2018-01-17 -0.11283  1318.0  1419.0          13.0           74.0
2018-01-18 -0.10986  1387.0  1311.0          11.0           12.0
2018-01-19 -0.10545  1265.0  1388.0          11.0           11.0
2018-01-22 -0.09887  1309.0  1327.0           9.0           13.0
2018-0

In [20]:
from sklearn.ensemble import RandomForestClassifier

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('------- RandomForestClassifier 30s -------')
print('PNL promedio: ', np.mean(result_RFC[:,0]))
print('buy promedio: ', np.mean(result_RFC[:,1]))
print('sell promedio: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_30s_RFC_wsize3_2018-01.csv')

------- RandomForestClassifier 30s -------
PNL promedio:  -0.100241
buy promedio:  1122.05
sell promedio:  1403.35
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.08839  1202.0  1330.0          12.0           32.0
2018-01-08 -0.09374  2033.0   491.0        1168.0           12.0
2018-01-09 -0.08801  2024.0   502.0        1555.0           12.0
2018-01-10 -0.10235  1259.0  1358.0          27.0           44.0
2018-01-11 -0.08663  1267.0  1340.0          12.0           15.0
2018-01-12 -0.12296   339.0  2287.0          11.0         1579.0
2018-01-15 -0.11561   302.0  2340.0          45.0         1560.0
2018-01-16 -0.09494  1206.0  1438.0          24.0           13.0
2018-01-17 -0.11051  1271.0  1466.0          11.0           91.0
2018-01-18 -0.10230  1439.0  1259.0          26.0           12.0
2018-01-19 -0.12804  1279.0  1374.0          17.0           11.0
2018-01-22 -0.09698  1283.0  1353.0          10.0           10.0
20

### ahora con intervalos de 40s

In [24]:
#Se lee la señal anteriormente procesada
data_set = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_40s_EURUSD-2018-01.csv", parse_dates=True,
                      infer_datetime_format=True, index_col=0)
window = 3
X, y_reg, bt = EU.build_dataset(data_set, window, binary_target=True, delete_constant_values=True, PNL=True)

In [25]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 40s ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/GaussianNB/pnl_40s_GNB_wsize3_2018-01.csv')

--------- GaussianNB 40s ----------------
average PNL :  -0.0775695
average buy:  827.35
average sell:  1091.95
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.07755   444.0  1482.0         303.0          939.0
2018-01-08 -0.06537  1837.0   105.0        1617.0           97.0
2018-01-09 -0.06115  1933.0     0.0        1933.0            0.0
2018-01-10 -0.06992  1729.0   252.0        1253.0          130.0
2018-01-11 -0.06184     0.0  2001.0           0.0         2001.0
2018-01-12 -0.09686  1846.0   151.0        1846.0          151.0
2018-01-15 -0.09233  2017.0     0.0        2017.0            0.0
2018-01-16 -0.07567     0.0  2000.0           0.0         2000.0
2018-01-17 -0.09090    13.0  2031.0          13.0         2031.0
2018-01-18 -0.08200   585.0  1451.0         174.0         1214.0
2018-01-19 -0.09276   698.0  1316.0         405.0         1027.0
2018-01-22 -0.08360   934.0  1088.0         612.0          316.0
2018-

In [26]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,4,1,1))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('-------- KNeighborsClassifier 40s -------')
print('average PNL: ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_40s_KNC_wsize3_2018-01.csv')

-------- KNeighborsClassifier 40s -------
average PNL:  -0.0721925
average buy:  853.05
average sell:  1066.25
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.07183   906.0  1020.0          14.0           14.0
2018-01-08 -0.06990  1576.0   366.0         902.0           11.0
2018-01-09 -0.06225  1597.0   336.0        1189.0           10.0
2018-01-10 -0.06764   890.0  1091.0          19.0           39.0
2018-01-11 -0.06449   906.0  1095.0          10.0           13.0
2018-01-12 -0.07116   272.0  1725.0           7.0         1192.0
2018-01-15 -0.08374   237.0  1780.0          50.0         1179.0
2018-01-16 -0.07703   866.0  1134.0          12.0           22.0
2018-01-17 -0.09937   976.0  1068.0          16.0           59.0
2018-01-18 -0.08091  1141.0   895.0          36.0           14.0
2018-01-19 -0.07265   977.0  1037.0          14.0           10.0
2018-01-22 -0.06833  1079.0   943.0          15.0           13.0
2018-0

In [27]:
from sklearn.tree import DecisionTreeClassifier

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,4,1,1))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('------ DecisionTreeClassifier 40s -------')
print('average PNL: ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_40s_DTC_wsize3_2018-01.csv')

------ DecisionTreeClassifier 40s -------
average PNL:  -0.07226
average buy:  863.45
average sell:  1055.85
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.08115   888.0  1038.0          10.0           14.0
2018-01-08 -0.07390   426.0  1516.0          10.0          884.0
2018-01-09 -0.06182  1607.0   326.0        1189.0            7.0
2018-01-10 -0.07283   983.0   998.0          18.0           35.0
2018-01-11 -0.07467   962.0  1039.0          12.0           13.0
2018-01-12 -0.06868   302.0  1695.0          11.0         1191.0
2018-01-15 -0.08910  1527.0   490.0        1168.0           48.0
2018-01-16 -0.06890   984.0  1016.0          23.0           16.0
2018-01-17 -0.08375  1017.0  1027.0          37.0           14.0
2018-01-18 -0.07357  1045.0   991.0          49.0           11.0
2018-01-19 -0.07160   939.0  1075.0           9.0           20.0
2018-01-22 -0.07102  1021.0  1001.0           9.0           17.0
2018-01-

In [28]:
from sklearn.ensemble import RandomForestClassifier

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('------- RandomForestClassifier 40s -------')
print('PNL promedio: ', np.mean(result_RFC[:,0]))
print('buy promedio: ', np.mean(result_RFC[:,1]))
print('sell promedio: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_40s_RFC_wsize3_2018-01.csv')

------- RandomForestClassifier 40s -------
PNL promedio:  -0.077048
buy promedio:  856.35
sell promedio:  1062.95
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.07573   895.0  1031.0          11.0           11.0
2018-01-08 -0.06861  1568.0   374.0         899.0           11.0
2018-01-09 -0.06662  1639.0   294.0        1190.0            8.0
2018-01-10 -0.07109   902.0  1079.0          18.0           47.0
2018-01-11 -0.06519   968.0  1033.0          13.0           13.0
2018-01-12 -0.09567   253.0  1744.0           7.0         1191.0
2018-01-15 -0.09083   223.0  1794.0          41.0         1185.0
2018-01-16 -0.08597   901.0  1099.0          24.0           17.0
2018-01-17 -0.07075   984.0  1060.0          14.0           61.0
2018-01-18 -0.07848  1052.0   984.0          20.0           11.0
2018-01-19 -0.09276   950.0  1064.0          10.0           11.0
2018-01-22 -0.07105  1060.0   962.0          11.0           12.0
201

### se probará con muestreo de 50s

In [29]:
#Se lee la señal anteriormente procesada
data_set = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_50s_EURUSD-2018-01.csv", parse_dates=True,
                      infer_datetime_format=True, index_col=0)
window = 3
X, y_reg, bt = EU.build_dataset(data_set, window, binary_target=True, delete_constant_values=True, PNL=True)

In [30]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 50s ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/GaussianNB/pnl_50s_GNB_wsize3_2018-01.csv')

--------- GaussianNB 50s ----------------
average PNL :  -0.06148
average buy:  443.157894737
average sell:  1109.73684211
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.05220  1489.0    84.0        1302.0           80.0
2018-01-09 -0.05239   479.0  1114.0         255.0          981.0
2018-01-10 -0.05777     0.0  1613.0           0.0         1613.0
2018-01-11 -0.04908     0.0  1614.0           0.0         1614.0
2018-01-12 -0.04926     0.0  1612.0           0.0         1612.0
2018-01-15 -0.07471  1630.0     0.0        1630.0            0.0
2018-01-16 -0.06034     0.0  1606.0           0.0         1606.0
2018-01-17 -0.07393     0.0  1653.0           0.0         1653.0
2018-01-18 -0.06269   240.0  1410.0         134.0         1216.0
2018-01-19 -0.07762   486.0  1154.0         317.0          863.0
2018-01-22 -0.06973   698.0   930.0         474.0          253.0
2018-01-23 -0.05871   198.0  1422.0         127.0          

In [31]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,4,1,1))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('-------- KNeighborsClassifier 50s -------')
print('average PNL: ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_50s_KNC_wsize3_2018-01.csv')

-------- KNeighborsClassifier 50s -------
average PNL:  -0.0574368421053
average buy:  674.210526316
average sell:  878.684210526
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.05458  1253.0   320.0         715.0           12.0
2018-01-09 -0.05240  1291.0   302.0         959.0            8.0
2018-01-10 -0.05394   661.0   952.0          14.0           35.0
2018-01-11 -0.05625   713.0   901.0          10.0           12.0
2018-01-12 -0.05248   184.0  1428.0           7.0         1028.0
2018-01-15 -0.06753    95.0  1535.0          32.0         1179.0
2018-01-16 -0.05528   819.0   787.0          37.0           15.0
2018-01-17 -0.06427   739.0   914.0          12.0           56.0
2018-01-18 -0.06230   915.0   735.0          37.0           17.0
2018-01-19 -0.07202   786.0   854.0          14.0           11.0
2018-01-22 -0.06122   862.0   766.0          16.0            9.0
2018-01-23 -0.05635   732.0   888.0          16.0   

In [32]:
from sklearn.tree import DecisionTreeClassifier

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,4,1,1))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('------ DecisionTreeClassifier 50s -------')
print('average PNL: ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_50s_DTC_wsize3_2018-01.csv')

------ DecisionTreeClassifier 50s -------
average PNL:  -0.0591242105263
average buy:  692.368421053
average sell:  860.526315789
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.05323  1231.0   342.0         705.0           12.0
2018-01-09 -0.05177  1306.0   287.0         961.0            8.0
2018-01-10 -0.06212   704.0   909.0          15.0           34.0
2018-01-11 -0.05436   743.0   871.0          11.0            9.0
2018-01-12 -0.05247   268.0  1344.0          11.0          951.0
2018-01-15 -0.06441   158.0  1472.0          32.0          962.0
2018-01-16 -0.06139   853.0   753.0          25.0           19.0
2018-01-17 -0.06791   781.0   872.0          44.0           16.0
2018-01-18 -0.07215   806.0   844.0          10.0           12.0
2018-01-19 -0.06137   813.0   827.0          12.0           11.0
2018-01-22 -0.05242   818.0   810.0          12.0           13.0
2018-01-23 -0.04745   821.0   799.0          32.0   

In [33]:
from sklearn.ensemble import RandomForestClassifier

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('------- RandomForestClassifier 50s -------')
print('PNL promedio: ', np.mean(result_RFC[:,0]))
print('buy promedio: ', np.mean(result_RFC[:,1]))
print('sell promedio: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_50s_RFC_wsize3_2018-01.csv')

------- RandomForestClassifier 50s -------
PNL promedio:  -0.0622526315789
buy promedio:  633.052631579
sell promedio:  919.842105263
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.05859   348.0  1225.0          13.0          703.0
2018-01-09 -0.05550  1288.0   305.0         972.0            9.0
2018-01-10 -0.06195   669.0   944.0          15.0           31.0
2018-01-11 -0.06059   712.0   902.0           7.0           14.0
2018-01-12 -0.07809   227.0  1385.0          10.0          962.0
2018-01-15 -0.06768   141.0  1489.0          31.0          963.0
2018-01-16 -0.05848   803.0   803.0          22.0           12.0
2018-01-17 -0.06545   767.0   886.0          14.0           46.0
2018-01-18 -0.05616   857.0   793.0          27.0           12.0
2018-01-19 -0.06393   759.0   881.0          12.0            8.0
2018-01-22 -0.06124   805.0   823.0          11.0           13.0
2018-01-23 -0.06510   794.0   826.0          30.

### se prueba con resample = 1 minuto

In [34]:
#Se lee la señal anteriormente procesada
data_set = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_1T_EURUSD-2018-01.csv", parse_dates=True,
                      infer_datetime_format=True, index_col=0)
window = 3
X, y_reg, bt = EU.build_dataset(data_set, window, binary_target=True, delete_constant_values=True, PNL=True)

In [35]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 1m ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/GaussianNB/pnl_1m_GNB_wsize3_2018-01.csv')

--------- GaussianNB 1m ----------------
average PNL :  -0.0522663157895
average buy:  636.315789474
average sell:  665.947368421
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.04382  1237.0    91.0        1096.0           91.0
2018-01-09 -0.03772  1299.0    24.0         852.0            9.0
2018-01-10 -0.05551  1334.0    35.0         768.0           13.0
2018-01-11 -0.04805   827.0   546.0         827.0          546.0
2018-01-12 -0.03802    54.0  1303.0          54.0         1303.0
2018-01-15 -0.06492  1375.0     0.0        1375.0            0.0
2018-01-16 -0.05220     0.0  1357.0           0.0         1357.0
2018-01-17 -0.06477     1.0  1382.0           1.0         1377.0
2018-01-18 -0.05732   643.0   746.0         576.0          701.0
2018-01-19 -0.06391   713.0   657.0         390.0          613.0
2018-01-22 -0.06052   761.0   608.0         444.0          169.0
2018-01-23 -0.04966   172.0  1184.0         102.0   

In [36]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,4,1,1))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('-------- KNeighborsClassifier 1m -------')
print('average PNL: ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_1m_KNC_wsize3_2018-01.csv')

-------- KNeighborsClassifier 1m -------
average PNL:  -0.0476647368421
average buy:  544.368421053
average sell:  757.894736842
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.04699  1086.0   242.0         611.0            8.0
2018-01-09 -0.04563   202.0  1121.0          10.0          795.0
2018-01-10 -0.05491   595.0   774.0          17.0           85.0
2018-01-11 -0.04645   641.0   732.0          12.0           12.0
2018-01-12 -0.04161   224.0  1133.0          13.0          851.0
2018-01-15 -0.05653   137.0  1238.0          27.0          811.0
2018-01-16 -0.05444   701.0   656.0          23.0           17.0
2018-01-17 -0.05451   661.0   722.0          28.0           87.0
2018-01-18 -0.04876   793.0   596.0          20.0           12.0
2018-01-19 -0.05172   679.0   691.0          13.0           11.0
2018-01-22 -0.04833   747.0   622.0          14.0           10.0
2018-01-23 -0.04897   607.0   749.0          19.0    

In [37]:
from sklearn.tree import DecisionTreeClassifier

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,4,1,1))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('------ DecisionTreeClassifier 1m -------')
print('average PNL: ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_1m_DTC_wsize3_2018-01.csv')

------ DecisionTreeClassifier 1m -------
average PNL:  -0.0521205263158
average buy:  636.421052632
average sell:  665.842105263
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.05229   281.0  1047.0          10.0          598.0
2018-01-09 -0.04281  1075.0   248.0         797.0           10.0
2018-01-10 -0.04864   618.0   751.0          12.0           22.0
2018-01-11 -0.04872   675.0   698.0           8.0           11.0
2018-01-12 -0.06776  1101.0   256.0         793.0           21.0
2018-01-15 -0.06548  1057.0   318.0         806.0           48.0
2018-01-16 -0.05351   726.0   631.0          17.0           15.0
2018-01-17 -0.04815   705.0   678.0          38.0           15.0
2018-01-18 -0.04963   729.0   660.0          36.0           11.0
2018-01-19 -0.06197   686.0   684.0          11.0           10.0
2018-01-22 -0.04446   698.0   671.0          10.0            9.0
2018-01-23 -0.05773   613.0   743.0           8.0    

In [38]:
from sklearn.ensemble import RandomForestClassifier

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('------- RandomForestClassifier 1m -------')
print('PNL promedio: ', np.mean(result_RFC[:,0]))
print('buy promedio: ', np.mean(result_RFC[:,1]))
print('sell promedio: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_1m_RFC_wsize3_2018-01.csv')

------- RandomForestClassifier 1m -------
PNL promedio:  -0.0526326315789
buy promedio:  584.894736842
sell promedio:  717.368421053
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-08 -0.04668   314.0  1014.0          16.0          598.0
2018-01-09 -0.04378  1062.0   261.0         797.0           11.0
2018-01-10 -0.05164   611.0   758.0          14.0           28.0
2018-01-11 -0.05102   655.0   718.0          12.0            9.0
2018-01-12 -0.04798  1122.0   235.0         787.0           11.0
2018-01-15 -0.05467   112.0  1263.0          26.0          813.0
2018-01-16 -0.05896   642.0   715.0          16.0           21.0
2018-01-17 -0.05105   642.0   741.0          12.0           38.0
2018-01-18 -0.04737   744.0   645.0          33.0           11.0
2018-01-19 -0.06381   666.0   704.0          12.0            9.0
2018-01-22 -0.05211   724.0   645.0          11.0           10.0
2018-01-23 -0.04724   649.0   707.0          12.0

### ahora se prueba con las muestras de 1 minuto y 30s

In [39]:
#Se lee la señal anteriormente procesada
data_set = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_1T30s_EURUSD-2018-01.csv", parse_dates=True,
                      infer_datetime_format=True, index_col=0)
window = 3
X, y_reg, bt = EU.build_dataset(data_set, window, binary_target=True, delete_constant_values=True, PNL=True)

In [40]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 1m30s ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/GaussianNB/pnl_1m30s_GNB_wsize3_2018-01.csv')

--------- GaussianNB 1m30s ----------------
average PNL :  -0.0361521052632
average buy:  364.210526316
average sell:  515.473684211
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells
2018-01-08 -0.02756  869.0   30.0         743.0           17.0
2018-01-09 -0.03024  368.0  542.0         324.0          537.0
2018-01-10 -0.03776  867.0   54.0         495.0           48.0
2018-01-11 -0.02322    0.0  910.0           0.0          910.0
2018-01-12 -0.04131  516.0  398.0         516.0          398.0
2018-01-15 -0.04457  910.0    0.0         910.0            0.0
2018-01-16 -0.03489  924.0    0.0         924.0            0.0
2018-01-17 -0.04715    3.0  940.0           3.0          939.0
2018-01-18 -0.03858  364.0  569.0         305.0          495.0
2018-01-19 -0.04781  416.0  514.0         260.0          431.0
2018-01-22 -0.04093  451.0  477.0         302.0          138.0
2018-01-23 -0.03572  157.0  765.0          75.0          356.0
2018-01-24

In [41]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,4,1,1))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('-------- KNeighborsClassifier 1m30s -------')
print('average PNL: ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_1m30s_KNC_wsize3_2018-01.csv')

-------- KNeighborsClassifier 1m30s -------
average PNL:  -0.0330778947368
average buy:  387.0
average sell:  492.684210526
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells
2018-01-08 -0.02738  739.0  160.0         415.0            8.0
2018-01-09 -0.03385  705.0  205.0         539.0           13.0
2018-01-10 -0.04704  337.0  584.0          21.0           54.0
2018-01-11 -0.03059  394.0  516.0          17.0           26.0
2018-01-12 -0.02650  138.0  776.0          11.0          574.0
2018-01-15 -0.03728  100.0  810.0          17.0          667.0
2018-01-16 -0.02353  511.0  413.0          28.0           19.0
2018-01-17 -0.05093  439.0  504.0          15.0           60.0
2018-01-18 -0.03260  592.0  341.0          38.0            9.0
2018-01-19 -0.03925  476.0  454.0          15.0           10.0
2018-01-22 -0.03211  458.0  470.0           9.0           18.0
2018-01-23 -0.03994  349.0  573.0           8.0           64.0
2018-01-24 -0.02469

In [42]:
from sklearn.tree import DecisionTreeClassifier

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,4,1,1))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('------ DecisionTreeClassifier 1m30s -------')
print('average PNL: ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_1m30s_DTC_wsize3_2018-01.csv')

------ DecisionTreeClassifier 1m30s -------
average PNL:  -0.0319463157895
average buy:  388.263157895
average sell:  491.421052632
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells
2018-01-08 -0.03758  209.0  690.0           9.0          401.0
2018-01-09 -0.02862  699.0  211.0         538.0            9.0
2018-01-10 -0.03859  377.0  544.0          11.0           26.0
2018-01-11 -0.03262  394.0  516.0          10.0            9.0
2018-01-12 -0.02109  141.0  773.0          20.0          537.0
2018-01-15 -0.03749   89.0  821.0          17.0          535.0
2018-01-16 -0.04022  477.0  447.0          28.0           13.0
2018-01-17 -0.03801  462.0  481.0          26.0           16.0
2018-01-18 -0.03206  514.0  419.0          24.0            9.0
2018-01-19 -0.03526  465.0  465.0          11.0           11.0
2018-01-22 -0.04175  447.0  481.0          10.0           12.0
2018-01-23 -0.03705  404.0  518.0           8.0           19.0
2018-01-24 

In [43]:
from sklearn.ensemble import RandomForestClassifier

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('------- RandomForestClassifier 1m30s -------')
print('PNL promedio: ', np.mean(result_RFC[:,0]))
print('buy promedio: ', np.mean(result_RFC[:,1]))
print('sell promedio: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_1m30s_RFC_wsize3_2018-01.csv')

------- RandomForestClassifier 1m30s -------
PNL promedio:  -0.0356573684211
buy promedio:  388.421052632
sell promedio:  491.263157895
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells
2018-01-08 -0.03441  720.0  179.0         417.0            9.0
2018-01-09 -0.03291  703.0  207.0         538.0           14.0
2018-01-10 -0.03361  354.0  567.0           9.0           35.0
2018-01-11 -0.02416  396.0  514.0          11.0           12.0
2018-01-12 -0.05451  146.0  768.0          20.0          537.0
2018-01-15 -0.04072  107.0  803.0          17.0          535.0
2018-01-16 -0.02622  494.0  430.0          35.0           11.0
2018-01-17 -0.03195  421.0  522.0          13.0           35.0
2018-01-18 -0.04084  566.0  367.0          33.0            8.0
2018-01-19 -0.03055  482.0  448.0          10.0            9.0
2018-01-22 -0.03527  498.0  430.0           9.0           11.0
2018-01-23 -0.03541  344.0  578.0           9.0           37.0
2018-01

### se prueba con los datos muestreados en intervalo de 5 min

In [44]:
#Se lee la señal anteriormente procesada
data_set = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_5T_EURUSD-2018-01.csv", parse_dates=True,
                      infer_datetime_format=True, index_col=0)
window = 3
X, y_reg, bt = EU.build_dataset(data_set, window, binary_target=True, delete_constant_values=True, PNL=True)

In [45]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 5m ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/GaussianNB/pnl_5m_GNB_wsize3_2018-01.csv')

--------- GaussianNB 5m ----------------
average PNL :  -0.0123530434783
average buy:  132.391304348
average sell:  89.7826086957
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells
2018-01-06 -0.00121    1.0    0.0           1.0            0.0
2018-01-08 -0.00512  260.0   19.0         228.0           19.0
2018-01-09 -0.00241  277.0    6.0         186.0            3.0
2018-01-10 -0.01532  217.0   64.0         148.0           64.0
2018-01-11 -0.01248  155.0  127.0         107.0          108.0
2018-01-12 -0.01715  163.0  117.0         159.0          112.0
2018-01-13 -0.00125    1.0    0.0           1.0            0.0
2018-01-15 -0.01813  280.0    0.0         280.0            0.0
2018-01-16 -0.01097    0.0  282.0           0.0          282.0
2018-01-17 -0.02044   38.0  250.0          21.0          119.0
2018-01-18 -0.02444  269.0   13.0         196.0           10.0
2018-01-19 -0.01747  150.0  133.0          80.0          127.0
2018-01-20 -0

In [46]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,4,1,1))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('-------- KNeighborsClassifier 5m -------')
print('average PNL: ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_5m_KNC_wsize3_2018-01.csv')

-------- KNeighborsClassifier 5m -------
average PNL:  -0.00713304347826
average buy:  109.826086957
average sell:  112.347826087
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells
2018-01-06 -0.00121    1.0    0.0           1.0            0.0
2018-01-08 -0.00526  228.0   51.0         171.0           13.0
2018-01-09 -0.00635  254.0   29.0         167.0            4.0
2018-01-10 -0.00996  135.0  146.0          21.0           37.0
2018-01-11 -0.00041  101.0  181.0           7.0           21.0
2018-01-12 -0.00272   20.0  260.0           6.0          175.0
2018-01-13 -0.00125    1.0    0.0           1.0            0.0
2018-01-15 -0.02048  236.0   44.0         168.0            7.0
2018-01-16 -0.00350  145.0  137.0          23.0           14.0
2018-01-17 -0.00734  160.0  128.0          14.0           28.0
2018-01-18 -0.01356  205.0   77.0          26.0            4.0
2018-01-19 -0.01470  145.0  138.0           8.0           11.0
2018-01-20  0

In [47]:
from sklearn.tree import DecisionTreeClassifier

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,4,1,1))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('------ DecisionTreeClassifier 5m -------')
print('average PNL: ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_5m_DTC_wsize3_2018-01.csv')

------ DecisionTreeClassifier 5m -------
average PNL:  -0.00761739130435
average buy:  107.608695652
average sell:  114.565217391
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells
2018-01-06 -0.00057    0.0    1.0           0.0            1.0
2018-01-08 -0.01204   66.0  213.0           8.0          126.0
2018-01-09 -0.00670  239.0   44.0         166.0            5.0
2018-01-10 -0.00613  129.0  152.0          14.0           19.0
2018-01-11 -0.01939  125.0  157.0           7.0            9.0
2018-01-12 -0.02653  224.0   56.0         174.0            7.0
2018-01-13 -0.00125    1.0    0.0           1.0            0.0
2018-01-15 -0.01328   63.0  217.0          14.0          168.0
2018-01-16 -0.00942  167.0  115.0          25.0           10.0
2018-01-17 -0.01092  149.0  139.0           7.0           21.0
2018-01-18 -0.00804  117.0  165.0          10.0           27.0
2018-01-19 -0.00373  148.0  135.0           8.0            5.0
2018-01-20  0

In [48]:
from sklearn.ensemble import RandomForestClassifier

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('------- RandomForestClassifier 5m -------')
print('PNL promedio: ', np.mean(result_RFC[:,0]))
print('buy promedio: ', np.mean(result_RFC[:,1]))
print('sell promedio: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_5m_RFC_wsize3_2018-01.csv')

------- RandomForestClassifier 5m -------
PNL promedio:  -0.00819956521739
buy promedio:  104.086956522
sell promedio:  118.086956522
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells
2018-01-06 -0.00057    1.0    0.0           1.0            0.0
2018-01-08 -0.01504  221.0   58.0         127.0           16.0
2018-01-09 -0.00762  225.0   58.0         166.0            7.0
2018-01-10 -0.01204  119.0  162.0          21.0           16.0
2018-01-11 -0.00830  123.0  159.0           6.0           10.0
2018-01-12 -0.02214   28.0  252.0           3.0          173.0
2018-01-13 -0.00138    1.0    0.0           1.0            0.0
2018-01-15 -0.00586  236.0   44.0         167.0           15.0
2018-01-16 -0.01119  169.0  113.0          32.0           11.0
2018-01-17 -0.01054  146.0  142.0          11.0           24.0
2018-01-18 -0.00153  166.0  116.0          26.0            9.0
2018-01-19 -0.00691  132.0  151.0           8.0           15.0
2018-01-2